# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [2]:
%cd "/Users/krishansingh/Documents/algorithmic-trading-python-master/starter_files/"

/Users/krishansingh/Documents/algorithmic-trading-python-master/starter_files


In [3]:
!pwd


/Users/krishansingh/Documents/algorithmic-trading-python-master/starter_files


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [9]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN='Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

It's now time to make the first version of our momentum screener!


We need to get one-year price returns for each stock in the universe. Here's how.

In [23]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3023960307261,
 'week52high': 188.4,
 'week52low': 116.74,
 'week52highSplitAdjustOnly': 183.62,
 'week52lowSplitAdjustOnly': 116.95,
 'week52change': 0.36137092169441604,
 'sharesOutstanding': 17093123279,
 'float': 0,
 'avg10Volume': 129826620,
 'avg30Volume': 128012912,
 'day200MovingAvg': 150.84,
 'day50MovingAvg': 165.49,
 'employees': 149960,
 'ttmEPS': 11.45,
 'ttmDividendRate': 0.8730540861864485,
 'dividendYield': 0.004996165362017835,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-24',
 'nextEarningsDate': '2022-01-17',
 'peRatio': 15.2347049457594,
 'beta': 1.382069928696359,
 'maxChangePercent': 69.14950913404229,
 'year5ChangePercent': 5.508073704753053,
 'year2ChangePercent': 1.57685372753872,
 'year1ChangePercent': 0.3549944513865095,
 'ytdChangePercent': 0.3450365310149412,
 'month6ChangePercent': 0.3240640788891517,
 'month3ChangePercent': 0.21181030768128092,
 'month1ChangePercent': 0.0935762997056258,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [13]:
data['year1ChangePercent']

0.3541723604351638

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
        pd.Series([symbol, data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'], 'N/A'],index=my_columns), ignore_index=True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,163.28,0.346474,N/A
1,AAL,19.07,0.188034,N/A
2,AAP,240.00,0.50148,N/A
3,AAPL,183.24,0.348057,N/A
4,ABBV,135.96,0.359946,N/A
...,...,...,...,...
500,YUM,140.21,0.28143,N/A
501,ZBH,129.23,-0.168867,N/A
502,ZBRA,601.83,0.533213,N/A
503,ZION,64.73,0.522444,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [34]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True)
df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.310,2.30108,N/A
1,148,DVN,43.530,2.00459,N/A
2,317,MRO,17.000,1.62872,N/A
3,299,MCHP,87.130,1.55616,N/A
4,175,FANG,108.460,1.51707,N/A
5,174,F,20.927,1.34552,N/A
6,195,FTNT,361.680,1.31635,N/A
7,345,NVDA,306.370,1.26453,N/A
8,344,NUE,118.020,1.20626,N/A
9,251,IT,337.100,1.11496,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [39]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input("Enter the value of your portflio")

    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number\n")
        portfolio_size=input("Enter the value of your portflio")

portfolio_input()
print(portfolio_size)

Enter the value of your portflio10000000
10000000


In [41]:
position_size = float(portfolio_size)/len(df.index)
for i in range(0,len(df.index)):
    df.loc[i,'Number of shares to buy']=math.floor(position_size/ df.loc[i,'Price'])

df

/Users/krishansingh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number of shares to buy
0,275,LB,80.310,2.30108,N/A,2490.0
1,148,DVN,43.530,2.00459,N/A,4594.0
2,317,MRO,17.000,1.62872,N/A,11764.0
3,299,MCHP,87.130,1.55616,N/A,2295.0
4,175,FANG,108.460,1.51707,N/A,1843.0
5,174,F,20.927,1.34552,N/A,9557.0
6,195,FTNT,361.680,1.31635,N/A,552.0
7,345,NVDA,306.370,1.26453,N/A,652.0
8,344,NUE,118.020,1.20626,N/A,1694.0
9,251,IT,337.100,1.11496,N/A,593.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [83]:
hqm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
    ]
hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
        pd.Series(
            [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A', 
            'N/A'
            ], 
            index=hqm_columns),
            ignore_index=True
        )
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,162.60,N/A,0.353078,N/A,0.0750975,N/A,-0.0903786,N/A,-0.0384788,N/A,N/A
1,AAL,18.86,N/A,0.182641,N/A,-0.178929,N/A,-0.112461,N/A,-0.0731443,N/A,N/A
2,AAP,240.07,N/A,0.4884,N/A,0.180376,N/A,0.108349,N/A,0.00848052,N/A,N/A
3,AAPL,181.85,N/A,0.355794,N/A,0.328918,N/A,0.215662,N/A,0.0926828,N/A,N/A
4,ABBV,135.72,N/A,0.358359,N/A,0.181581,N/A,0.262374,N/A,0.148494,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.05,N/A,0.277928,N/A,0.154683,N/A,0.0770819,N/A,0.071218,N/A,N/A
501,ZBH,128.02,N/A,-0.175171,N/A,-0.260284,N/A,-0.147638,N/A,-0.0438639,N/A,N/A
502,ZBRA,610.17,N/A,0.538002,N/A,0.159565,N/A,0.0548217,N/A,-0.0282264,N/A,N/A
503,ZION,63.20,N/A,0.529535,N/A,0.191589,N/A,0.0948515,N/A,-0.0673197,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [65]:
hqm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         505 non-null    object 
 1   Price                          505 non-null    float64
 2   Number of Shares to Buy        505 non-null    object 
 3   One-Year Price Return          501 non-null    object 
 4   One-Year Return Percentile     505 non-null    object 
 5   Six-Month Price Return         501 non-null    object 
 6   Six-Month Return Percentile    505 non-null    object 
 7   Three-Month Price Return       501 non-null    object 
 8   Three-Month Return Percentile  505 non-null    object 
 9   One-Month Price Return         501 non-null    object 
 10  One-Month Return Percentile    505 non-null    object 
 11  HQM Score                      505 non-null    object 
dtypes: float64(1), object(11)
memory usage: 47.5+ KB


In [84]:
time_periods =[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        if hqm_df.loc[row, f'{time_period} Price Return'] == None:
            hqm_df.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100


hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,162.60,N/A,0.353078,0.619802,0.0750975,0.514851,-0.0903786,0.10099,-0.0384788,0.257426,N/A
1,AAL,18.86,N/A,0.182641,0.380198,-0.178929,0.0693069,-0.112461,0.0732673,-0.0731443,0.10297,N/A
2,AAP,240.07,N/A,0.4884,0.786139,0.180376,0.756436,0.108349,0.70495,0.00848052,0.548515,N/A
3,AAPL,181.85,N/A,0.355794,0.627723,0.328918,0.938614,0.215662,0.914851,0.0926828,0.920792,N/A
4,ABBV,135.72,N/A,0.358359,0.629703,0.181581,0.760396,0.262374,0.954455,0.148494,0.984158,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.05,N/A,0.277928,0.522772,0.154683,0.679208,0.0770819,0.580198,0.071218,0.881188,N/A
501,ZBH,128.02,N/A,-0.175171,0.0316832,-0.260284,0.0277228,-0.147638,0.029703,-0.0438639,0.223762,N/A
502,ZBRA,610.17,N/A,0.538002,0.831683,0.159565,0.693069,0.0548217,0.49505,-0.0282264,0.306931,N/A
503,ZION,63.20,N/A,0.529535,0.825743,0.191589,0.784158,0.0948515,0.651485,-0.0673197,0.126733,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [85]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row,'HQM Score']=mean(momentum_percentiles)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,162.60,N/A,0.353078,0.619802,0.0750975,0.514851,-0.0903786,0.10099,-0.0384788,0.257426,0.373267
1,AAL,18.86,N/A,0.182641,0.380198,-0.178929,0.0693069,-0.112461,0.0732673,-0.0731443,0.10297,0.156436
2,AAP,240.07,N/A,0.4884,0.786139,0.180376,0.756436,0.108349,0.70495,0.00848052,0.548515,0.69901
3,AAPL,181.85,N/A,0.355794,0.627723,0.328918,0.938614,0.215662,0.914851,0.0926828,0.920792,0.850495
4,ABBV,135.72,N/A,0.358359,0.629703,0.181581,0.760396,0.262374,0.954455,0.148494,0.984158,0.832178
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.05,N/A,0.277928,0.522772,0.154683,0.679208,0.0770819,0.580198,0.071218,0.881188,0.665842
501,ZBH,128.02,N/A,-0.175171,0.0316832,-0.260284,0.0277228,-0.147638,0.029703,-0.0438639,0.223762,0.0782178
502,ZBRA,610.17,N/A,0.538002,0.831683,0.159565,0.693069,0.0548217,0.49505,-0.0282264,0.306931,0.581683
503,ZION,63.20,N/A,0.529535,0.825743,0.191589,0.784158,0.0948515,0.651485,-0.0673197,0.126733,0.59703


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [86]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,74.80,N/A,1.03063,0.980198,0.415461,0.976238,0.431271,0.992079,0.108473,0.942574,0.972772
1,PFE,60.90,N/A,0.687686,0.930693,0.538291,0.986139,0.379857,0.984158,0.16933,0.990099,0.972772
2,ANET,148.39,N/A,0.961087,0.974257,0.54561,0.988119,0.594914,0.99802,0.0916792,0.918812,0.969802
3,LB,82.85,N/A,2.29621,1,0.82224,0.99802,0.222357,0.926733,0.080805,0.90297,0.956931
4,EXR,219.00,N/A,0.993834,0.978218,0.377977,0.968317,0.235933,0.938614,0.105454,0.938614,0.955941
5,AVGO,690.70,N/A,0.557842,0.845545,0.446518,0.982178,0.326741,0.968317,0.198574,0.994059,0.947525
6,HPQ,38.87,N/A,0.592512,0.877228,0.301282,0.928713,0.375322,0.982178,0.175908,0.992079,0.94505
7,DVN,43.25,N/A,2.03986,0.99802,0.555682,0.990099,0.431623,0.994059,0.0478581,0.774257,0.939109
8,PLD,164.57,N/A,0.722841,0.946535,0.342678,0.950495,0.256545,0.950495,0.0831491,0.90495,0.938119
9,AZO,2088.70,N/A,0.733235,0.950495,0.419624,0.978218,0.202475,0.89505,0.0858109,0.908911,0.933168


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [77]:
portfolio_input()

Enter the value of your portflio10000000


In [87]:
position_size=float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy']=math.floor(position_size)/hqm_df.loc[i, 'Price']

hqm_df

/Users/krishansingh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,74.80,2673.8,1.03063,0.980198,0.415461,0.976238,0.431271,0.992079,0.108473,0.942574,0.972772
1,PFE,60.90,3284.07,0.687686,0.930693,0.538291,0.986139,0.379857,0.984158,0.16933,0.990099,0.972772
2,ANET,148.39,1347.8,0.961087,0.974257,0.54561,0.988119,0.594914,0.99802,0.0916792,0.918812,0.969802
3,LB,82.85,2414,2.29621,1,0.82224,0.99802,0.222357,0.926733,0.080805,0.90297,0.956931
4,EXR,219.00,913.242,0.993834,0.978218,0.377977,0.968317,0.235933,0.938614,0.105454,0.938614,0.955941
5,AVGO,690.70,289.561,0.557842,0.845545,0.446518,0.982178,0.326741,0.968317,0.198574,0.994059,0.947525
6,HPQ,38.87,5145.36,0.592512,0.877228,0.301282,0.928713,0.375322,0.982178,0.175908,0.992079,0.94505
7,DVN,43.25,4624.28,2.03986,0.99802,0.555682,0.990099,0.431623,0.994059,0.0478581,0.774257,0.939109
8,PLD,164.57,1215.29,0.722841,0.946535,0.342678,0.950495,0.256545,0.950495,0.0831491,0.90495,0.938119
9,AZO,2088.70,95.7533,0.733235,0.950495,0.419624,0.978218,0.202475,0.89505,0.0858109,0.908911,0.933168


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [88]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='momentum_strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [89]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [94]:
column_formats = {
    'A':['Ticker',string_template], 
    'B':['Price',dollar_template], 
    'C':['Number of Shares to Buy',integer_template], 
    'D':['One-Year Price Return',percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['momentum_strategy'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['momentum_strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [95]:
 writer.save()